In [ ]:
from dl.simple_cnn import SimpleCNN
from dl.shuffle_net import ShuffleNetV2
from dl.mobile_net import MobileNET
from dl.wide_resnet import WideResNet
from dl.capsule_net import CapsuleNet
from dl.callbacks.plot_loss import PlotLosses
from data.data_set import get_data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN, EarlyStopping, ModelCheckpoint, TensorBoard
from config import DL_MODELS_PATH, TB_LOGS_PATH
import os
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from swa.tfkeras import SWA
from dl.callbacks.cyclical_lr import CyclicLR
from plotly.offline import init_notebook_mode
from visualization.visualize_history import plot_accuracy_and_loss

init_notebook_mode(connected=True)

In [ ]:
((X_train, y_train), (X_val, y_val)) = get_data()

In [ ]:
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range= .8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = .1, # Randomly zoom image 
        width_shift_range= .1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range= .1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=.05,  # randomly flip images horizontally
        vertical_flip=False)  # randomly flip images vertically

In [ ]:
def get_callbacks(model_name, use_cyclic_lr=False, plot=False):
    callbacks = [TerminateOnNaN(), EarlyStopping(monitor='val_accuracy', patience=5)]
    callbacks.append(ModelCheckpoint(os.path.join(DL_MODELS_PATH, model_name + '-{epoch:02d}-{val_accuracy:.2f}.hdf5'), monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max'))
    tb_path = os.path.join(TB_LOGS_PATH, model_name)
    pathlib.Path(tb_path).mkdir(parents=True, exist_ok=True)
    callbacks.append(TensorBoard(log_dir=tb_path))
    callbacks.append(SWA(start_epoch=5, lr_schedule='constant', swa_lr=0.001, verbose=1))
    if plot:
        callbacks.append(PlotLosses())
    if use_cyclic_lr:
        callbacks.append(CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.))
    else:
        rlronp =  ReduceLROnPlateau(monitor='val_loss',
                                    patience=3,
                                    verbose=1,
                                    factor=0.5,
                                    min_lr=0.00005)
        
        callbacks.append(rlronp)
    return callbacks

In [ ]:
epochs = 100
Learning_rate = 0.001
decay= 5 * Learning_rate / epochs
optimizers = {'adam': Adam(lr=Learning_rate, decay= 3 * Learning_rate / epochs),
              'rmsprop': RMSprop(lr=Learning_rate, rho=0.9, epsilon=1e-08, decay= 0)}

In [ ]:
epochs = 50
histories = {}
models = {}

for opt_name, opt in optimizers.items():
    # Train capsule net
    model_name = f'capsnet_{opt_name}'
    model = WideResNet(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          32, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    # Train a basic cnn
    model_name = f'simple_cnn_{opt_name}'
    model = SimpleCNN(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          2048, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    # Train shufflenetv2
    model_name = f'shuffle_netv2_{opt_name}'
    model = SimpleCNN(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          256, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    # Train efficient net
    model_name = f'wide_resnet_{opt_name}'
    model = WideResNet(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          16, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    # Train mobile_net
    model_name = f'mobile_net_{opt_name}'
    model = MobileNET(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          32, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model

In [ ]:
plot_accuracy_and_loss(histories)